### Imports

In [ ]:
import matplotlib.pyplot as plt
import contextily as cx
import geopandas as gpd
import os
import copy
import json
import pandas as pd
import numpy as np
import random
from matplotlib.lines import Line2D
from matplotlib import rcParams

from mnms.tools.render import draw_roads, draw_links_load
from mnms.io.graph import load_graph
from mnms.time import Time, Dt

### Global parameters

In [ ]:
rcParams['font.size'] = 16

# Roads typology visualization

### Parameters

In [ ]:
current_dir = os.getcwd()
rt_param_file_path = current_dir + '/param_roadstypo.json'
with open(rt_param_file_path, 'r') as rt_param_file:
    rt_param_json = json.load(rt_param_file)
rt_indir = rt_param_json['INPUT']['indir']
mlgraph_file = rt_indir + rt_param_json['INPUT']['network_file']
roads_typo = rt_param_json['GRAPH']['roads_typo']
primary_sections = roads_typo['primary'] # NB: here roads sections and mlgraph links have the same names
secondary_sections = roads_typo['secondary']

#lyon_iris_shp = current_dir + '/INPUTS/CONTOURS-IRIS.shp'
#iris_shp = gpd.read_file(lyon_iris_shp)
#lyon6_shp = iris_shp[iris_shp['NOM_COM'] == 'Lyon 6e Arrondissement']
#duquesne_we_garibaldi_ns = ['T_744067565_FRef', 'T_58229809_FRef', 'T_58229765_FRef', 'T_58229704_FRef', 'T_58229665_FRef', 'T_58229620_FRef', 'T_58229589_FRef', 'T_58229552_FRef', 'T_58229522_FRef', 'T_58229923_FRef', 'T_58230150_FRef', 'T_58230298_FRef', 'T_61618086_FRef', 'T_58230633_FRef', 'T_58230753_FRef', 'T_747119237_FRef', 'T_58231166_FRef', 'T_58231291_FRef', 'T_58231482_FRef', 'T_58231638_FRef', 'T_58231807_FRef', 'T_58231996_FRef', 'T_1035388886_FRef', 'T_1035388885_FRef', 'T_778807201_FRef', 'T_58232446_FRef']
#bvbelges_sn_duquesne_ew = ['T_744064732_FRef', 'T_744064733_FRef', 'T_740854577_FRef', 'T_740854572_FRef', 'T_740854573_FRef', 'T_58231869_FRef', 'T_58231678_FRef', 'T_58231544_FRef', 'T_62193589_FRef', 'T_62193588_FRef', 'T_704007182_FRef', 'T_58230917_FRef', 'T_58230834_FRef', 'T_58230761_FRef', 'T_58230560_FRef', 'T_58230295_FRef', 'T_58230102_FRef', 'T_58229991_FRef', 'T_58229906_FRef', 'T_58229742_FRef', 'T_58229593_FRef', 'T_744067569_FRef', 'T_744067568_FRef', 'T_58229522_toRef', 'T_58229552_toRef', 'T_58229589_toRef', 'T_58229620_toRef', 'T_58229665_toRef', 'T_58229704_toRef', 'T_58229765_toRef', 'T_58229809_toRef', 'T_744067565_toRef']

### Load and plot primary and secondary sections

In [ ]:
mlgraph = load_graph(mlgraph_file)

roads_secondary = copy.deepcopy(mlgraph.roads)
for lid in primary_sections:
    roads_secondary.delete_section(lid)
    
roads_primary = copy.deepcopy(mlgraph.roads)
for lid in mlgraph.roads.sections.keys():
    if lid not in primary_sections:
        roads_primary.delete_section(lid)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
draw_roads(ax, roads_secondary, color='grey', linkwidth=2, nodesize=0, draw_stops=False, node_label=False, label_size=1)
draw_roads(ax, roads_primary, color='red', linkwidth=3, nodesize=0, draw_stops=False, node_label=False, label_size=1)
draw_roads(ax, mlgraph.roads, color='grey', linkwidth=0, nodesize=3, draw_stops=False, node_label=False, label_size=1)
#cx.add_basemap(ax, crs=lyon6_shp.crs, source=cx.providers.OpenStreetMap.France, alpha=0.4)
plt.xticks([])
plt.yticks([])
legend = [Line2D([0, 1], [0, 1], color='red', linewidth=2, label='Primary links'),
         Line2D([0, 1], [0, 1], color='grey', linewidth=2, label='Secondary links')]
plt.legend(handles=legend)
#plt.savefig('lyon6_roads_typo.pdf', bbox_inches='tight')
plt.show()

### Create demand scenario to show effect of roads typology

In [ ]:
# Gather south-east and north-west nodes
central_node = [843875, 6520375]
se_nodes = [nid for nid,n in mlgraph.graph.nodes.items() if n.position[0] > central_node[0] and n.position[1] < central_node[1]]
nw_nodes = [nid for nid,n in mlgraph.graph.nodes.items() if n.position[0] < central_node[0] and n.position[1] > central_node[1]]

# Display them
fig, ax = plt.subplots(figsize=(10, 10))
draw_roads(ax, mlgraph.roads, color='grey', linkwidth=1, nodesize=0, draw_stops=False, node_label=False, label_size=1)

# South east nodes
x, y = zip(*[mlgraph.graph.nodes[n].position for n in se_nodes])
ax.plot(x, y, 'o', markerfacecolor='white', markeredgecolor='red', fillstyle='full', markersize=2)
# North west nodes
x, y = zip(*[mlgraph.graph.nodes[n].position for n in nw_nodes])
ax.plot(x, y, 'o', markerfacecolor='white', markeredgecolor='blue', fillstyle='full', markersize=2)
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
# Parameters
new_demand_file = rt_indir + '/demand_southeast_northwest.csv'
dep_rate = 5.5 # dep/s
tstart = Time('00:00:00')
tend = Time('00:30:00')
patterns = ['se->nw', 'nw->se']

# Users creation
t = 0
uid = 0
travelers = []
while True:
    t += random.expovariate(dep_rate)
    if tstart.add_time(Dt(seconds=t)) >= tend:
        break
    while True:
        # Choose a random pattern
        pattern = np.random.choice(patterns)
        # Choose a random OD for this pattern 
        if pattern == 'nw->se':
            on = np.random.choice(nw_nodes)
            dn = np.random.choice(se_nodes)
        elif pattern == 'se->nw':
            on = np.random.choice(se_nodes)
            dn = np.random.choice(nw_nodes)
        if on != dn:
            travelers.append(['U'+str(uid), 
                              tstart.add_time(Dt(seconds=round(t,0))), 
                              str(mlgraph.graph.nodes[on].position[0])+' '+str(mlgraph.graph.nodes[on].position[1]),
                              str(mlgraph.graph.nodes[dn].position[0])+' '+str(mlgraph.graph.nodes[dn].position[1])])
            uid += 1
            break
# Export to csv
df = pd.DataFrame(travelers, columns=['ID', 'DEPARTURE', 'ORIGIN', 'DESTINATION'])
df.to_csv(new_demand_file, sep=';', index=False)

NB: If you want to use this new demand set for comparison, replace the demand file name in the parameters json files and launch the simulations before launching the rest of the cells.

# Comparison with and without roads typology

### Parameters

In [ ]:
rt_outdir = rt_param_json['OUTPUT']['output_dir']
rt_user_file = rt_outdir + rt_param_json['OUTPUT']['user_file']

nrt_param_file_path = current_dir + '/param.json'
with open(nrt_param_file_path, 'r') as nrt_param_file:
    nrt_param_json = json.load(nrt_param_file)
nrt_outdir = nrt_param_json['OUTPUT']['output_dir']
nrt_user_file = nrt_outdir + nrt_param_json['OUTPUT']['user_file']

### Compare links load

In [ ]:
# Get users in road typo scenario
with open(rt_user_file) as f:
    dfrt = pd.read_csv(f, sep=';')
rt_users_ids = set(dfrt['ID'].tolist())

# Get users in non road typo scenario
with open(nrt_user_file) as f:
    dfnrt = pd.read_csv(f, sep=';')
nrt_users_ids = set(dfnrt['ID'].tolist())

In [ ]:
# Compute links load in road typo scenario
rt_links_load = {lid: 0 for lid, link in mlgraph.graph.links.items() if 'ORIGIN' not in link.upstream and 'DESTINATION' not in link.downstream}

for uid in rt_users_ids:
    dfrtu = dfrt[dfrt['ID'] == uid]
    linksu = [l for i,l in enumerate(dfrtu['LINK'].tolist()) if i == 0 or (i > 0 and l != dfrtu['LINK'].tolist()[i-1])]
    linksu = [l for l in linksu if str(l) != 'nan']
    for link in linksu:
        unode, dnode = link.split(' ')
        try:
            link_obj = mlgraph.graph.nodes[unode].adj[dnode]
            lid = link_obj.id
            rt_links_load[lid] += 1
        except:
            pass

## Compute links load in non road typo scenario
nrt_links_load = {lid: 0 for lid, link in mlgraph.graph.links.items() if 'ORIGIN' not in link.upstream and 'DESTINATION' not in link.downstream}

for uid in nrt_users_ids:
    dfnrtu = dfnrt[dfnrt['ID'] == uid]
    linksu = [l for i,l in enumerate(dfnrtu['LINK'].tolist()) if i == 0 or (i > 0 and l != dfnrtu['LINK'].tolist()[i-1])]
    linksu = [l for l in linksu if str(l) != 'nan']
    for link in linksu:
        unode, dnode = link.split(' ')
        try:
            link_obj = mlgraph.graph.nodes[unode].adj[dnode]
            lid = link_obj.id
            nrt_links_load[lid] += 1
        except:
            pass

In [ ]:
max_load = (max(max(rt_links_load.values()), max(nrt_links_load.values())))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
draw_links_load(ax1, mlgraph.graph, rt_links_load, 10, lmin=0, lmax=max_load)
draw_links_load(ax2, mlgraph.graph, nrt_links_load, 10, lmin=0, lmax=max_load)
ax1.set_xticks([])
ax2.set_xticks([])
ax1.set_yticks([])
ax2.set_yticks([])
ax1.set_title('With weights on road typology')
ax2.set_title('Without road typology')
im = plt.scatter([], [], cmap="cool", c=[], vmin=0, vmax=max_load)
fig.subplots_adjust(bottom=0.25, wspace=0.05)
cbar_ax = fig.add_axes([0.17, 0.15, 0.7, 0.05])
cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal')
cbar.ax.set_xlabel('Number of visits')
plt.show()
#plt.savefig('lyon6_comparison_road_typo.pdf', bbox_inches='tight')